In [1]:
'''
Improvements to do:
1. Clean data(index to date) before returning bars dataframe
2. DataHandler C : Pickle type for offline data
3. Automatically determine how many strategies used. Idea: From function generate_signals it will send +1
4. Merge Moving Average Strategies classes
'''

from classes.backtest import *
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

symbol = "AAPL"
start = datetime(2018,1,1)
end =  datetime(2019,1,1)
capital = 100000.0
#frequency = "daily"
#transaction_costs = 5

# Obtain daily data from chosen stock and data provider
bars = DataHandler("B", symbol, start, end).generate_data()

print(type(bars))
print(bars.head(10))

# Create a set of signals from chosen Strategies
sma = SimpleMAStrategy(symbol, bars, 20)
macross = MACrossStrategy(symbol, bars, 25, 50)
ema = EMAStrategy(symbol, bars, 20)
#macd = MAConDivStrategy(symbol, bars)
#dchian = DChianStrategy(symbol, bars)
#bb = BBStrategy(symbol, bars)
#buyhold = BuyHoldStrategy(symbol, bars)
signals1 = sma.generate_signals()
signals2 = macross.generate_signals()
signals3 = ema.generate_signals()

# Create a portfolio of chosen Stock with chosen initial capital
totalsignals = pd.DataFrame(index=bars.index)
#TODO- Automize
totalsignals["signal1"] = signals1.signal
totalsignals["signal2"] = signals2.signal
totalsignals["signal3"] = signals3.signal
weights = [0.1,0.15,0.75]
portfolio = MarketOnClosePortfolio(symbol, bars, totalsignals, weights, capital)


Generating data
option B selected. Pandas Datareader
<class 'pandas.core.frame.DataFrame'>
                open      high       low     close    volume
date                                                        
2018-01-02  167.6431  169.7514  166.7564  169.7120  25555934
2018-01-03  169.9780  171.9682  169.4165  169.6825  29517899
2018-01-04  169.9879  170.9041  169.5347  170.4707  22434597
2018-01-05  170.8746  172.7760  170.4904  172.4115  23660018
2018-01-08  171.7711  173.0125  171.3573  171.7711  20567766
2018-01-09  171.9682  172.4706  170.8450  171.7514  21583997
2018-01-10  170.5987  171.7219  170.4411  171.7120  23959895
2018-01-11  172.0076  172.8929  171.9091  172.6874  18667729
2018-01-12  173.5741  174.7366  173.0519  174.4706  25418080
2018-01-16  175.2686  176.7366  173.5346  173.5839  29565947


In [2]:
# Optimize strategies' weights
##Parameters
population = 15  #Population size. Number of fireflies
maxgen = 50   #MaxGen: the maximal number of generations
alpha = 0.2   #Initial value for randomization parameter
beta = 1   #Attractiveness metric
gamma = 1     #gamma: the light absorption coefficient 
dimension = 3 #Number of strategies to optimize

parameters = [population, maxgen, alpha, beta, gamma, dimension, totalsignals, weights]
firefly  = FireFly(parameters)


#portfolio.setWeights  = firefly.optimized_weights
#portfolio.generate_positions()

#returns = portfolio.backtest_portfolio()
#print(portfolio.totalsignals.head())
#print(portfolio.positions.head())
#print(returns.head(30))



Population:15
MaxGen:50
Alpha:0.2
Beta:1.0
Gamma:1.0
Dimension:3

Initialize fireflies
Dimension:0
Dimension:1
Dimension:2
Firefly agents Before
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
0.620106983662264
0.7484407185039168
0.5389099862745382
0.2771813620098692
0.03568694430937569
0.5434273306905196
0.1930531154781875
0.7342002958748693
0.7562706784911317
0.6295801407889285
0.8062863729046819
0.819424658022249
0.37511072230204023
0.7733681910438825
0.9959066704023887
0.6326990720532849
0.6552389817022821
0.023023496513150832
0.9945534494424241
0.08840616115158506
0.8908489515654999
0.16968476910484653
0.045806528296422844
0.5712619296325632
0.864197553926266
0.625247587783

In [ ]:
# Performance: Graphs

# Reset index and format dates column
bars = bars.reset_index()
bars['date'] = pd.to_datetime(bars['date'])
signals1 = signals1.reset_index()
signals1['date'] = pd.to_datetime(signals1['date'])
signals2 = signals2.reset_index()
signals2['date'] = pd.to_datetime(signals2['date'])
signals3 = signals3.reset_index()
signals3['date'] = pd.to_datetime(signals3['date'])

##plot bars
plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(12,8))
ax.plot_date(bars.date, bars["close"],'-',c='r')

##plot Technical indicators 
ax.plot(signals1.date,signals1["mavg"], c='y', label="100-days SMA")
#ax.plot(signals2.date,signals2["short_mavg"], '--', c='g',lw=1,label="MA 30")
#ax.plot(signals2.date,signals2["long_mavg"], '--',c='b',lw=1,label="MA 50")
ax.plot(signals3.date,signals3["emavg"], c='b', label="20-days EMA")

##plot positions
'''# Plot the strategy1 buy signals
ax.plot(signals1.loc[signals1.positions == 1.0].date, 
        signals1.mavg[signals1.positions == 1.0], '^', markersize=10, color='m',label="Buy")         
# Plot the strategy1 sell signals
ax.plot(signals1.loc[signals1.positions == -1.0].date, 
        signals1.mavg[signals1.positions == -1.0], 'v', markersize=10, color='k',label="Sell")'''
'''# Plot the strategy2 buy signals
ax.plot(signals2.loc[signals2.positions == 1.0].date, 
        signals2.short_mavg[signals2.positions == 1.0], '^', markersize=10, color='m',label="Buy")         
# Plot the strategy2 sell signals
ax.plot(signals2.loc[signals2.positions == -1.0].date, 
        signals2.short_mavg[signals2.positions == -1.0], 'v', markersize=10, color='k',label="Sell")'''

plt.title(symbol)# a title notation
plt.xlabel('Days') # adding xlabel
plt.ylabel('Prices') # adding ylabel
plt.legend(loc ='best', shadow = True, fancybox = False, title ='Legend', ncol =1)

fig.autofmt_xdate()
plt.tight_layout()
plt.show()

# Performance: Stats